In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()


## A Quick Test!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. 

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad you're here and excited to chat. How can I assist you today?


In [5]:
# A class to represent a Webpage
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [16]:
# Let's try one out. Change the website and add print statements to follow along.

volvo = Website("https://www.volvogroup.com/en/")
print(volvo.title)
print(volvo.text)

Home | Volvo Group
We noticed that you are using a browser that is no longer supported by our system, to enjoy the full Volvo experience consider changing browser
x
Search
Our brands
Contact us
Our brands
Contact us
Sustainable Transportation
Sustainable Solutions
Renewable fuels
Hydrogen fuel cells and combustion engines
Battery electric
CES 2025
The way forward
Collaborative Approach
The Mobility Transformation Forum
Innovation & Collaboration
Responsible Business
Careers
Students, graduates & internships
Working at Volvo Group
Recruitment focus areas
Job openings
#tecHER newsletter
Investors
Financial Reports & Presentations
Truck Orders and Deliveries
Key Ratios
Current IR Presentation
Financial Calendar
Contact Investor Relations
Subscribe for press releases
The Volvo Share
Share Price Calculator
Ownership Information
Analyst Coverage
Conversion of Shares
Dividend
Debt Information
Debt Programmes
Long-term committed facilities
Green financing
Financial Targets
Risks & Uncertaintie

In [17]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [18]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [19]:
print(user_prompt_for(volvo))

You are looking at a website titled Home | Volvo Group
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

We noticed that you are using a browser that is no longer supported by our system, to enjoy the full Volvo experience consider changing browser
x
Search
Our brands
Contact us
Our brands
Contact us
Sustainable Transportation
Sustainable Solutions
Renewable fuels
Hydrogen fuel cells and combustion engines
Battery electric
CES 2025
The way forward
Collaborative Approach
The Mobility Transformation Forum
Innovation & Collaboration
Responsible Business
Careers
Students, graduates & internships
Working at Volvo Group
Recruitment focus areas
Job openings
#tecHER newsletter
Investors
Financial Reports & Presentations
Truck Orders and Deliveries
Key Ratios
Current IR Presentation
Financial Calendar
Contact Investor Relations
Subscribe for press releases
The Volvo Share
Share

## The API from OpenAI expects to receive messages in a particular structure.
## Many of the other APIs share this structure:

In [15]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [20]:
# Try this out, and then try for a few more websites

messages_for(volvo)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home | Volvo Group\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nWe noticed that you are using a browser that is no longer supported by our system, to enjoy the full Volvo experience consider changing browser\nx\nSearch\nOur brands\nContact us\nOur brands\nContact us\nSustainable Transportation\nSustainable Solutions\nRenewable fuels\nHydrogen fuel cells and combustion engines\nBattery electric\nCES 2025\nThe way forward\nCollaborative Approach\nThe Mobility Transformation Forum\nInnovation & Collaboration\nResponsible Business\nCareers\nStudents, graduates & internships\nWorking at Volvo Group\nRecrui

## Time to bring it together - the API for OpenAI is very simple!

In [21]:
# And now: call the OpenAI API.

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [22]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [24]:
display_summary("https://www.volvogroup.com/en/")

# Summary of Volvo Group Website

The Volvo Group website provides comprehensive information about the company, which has a strong global presence with nearly 102,000 employees, 14 brands, and operations in approximately 180 markets. The Group focuses on innovative and sustainable transport solutions, encompassing products such as trucks, buses, construction equipment, and marine and industrial engines.

## Key Sections:

- **Sustainable Transportation**: The Volvo Group is committed to sustainable solutions, including renewable fuels, hydrogen technology, and battery electric options.
- **Products and Services**: Offers an exploration of the various brands and services under the Volvo umbrella.
- **Corporate Information**: Details about corporate governance, financial performance, and investor relations.
- **Careers**: Opportunities for students, graduates, and job seekers, along with information about working at Volvo Group.

## Latest News Highlights:

1. **Passing of Andrea Fuder**: The Chief Purchasing Officer and Executive Board member passed away after a short illness. She was known for her leadership in transforming the supply chain industry during challenging times.
  
2. **Assar Gabrielsson Prize**: Awards were given to Marcus Borenäs and Elizabeth Schepke for their contributions in research, each receiving SEK 100,000 for their work.
  
3. **First Quarter Report (2025)**: A decline in net sales by 7% due to global trade uncertainties, but growth in the service business was observed. 

4. **Annual General Meeting**: Held on April 2, 2025, at which company leadership presented operational insights.

5. **Update on Shares**: Recent changes in share voting rights due to the conversion of Series A shares to Series B. 

Overall, the website highlights Volvo Group's advancements in sustainable transport, financial transparency, and commitment to corporate responsibility.

In [25]:
display_summary("https://www.ericsson.com/en")

# Summary of "Enabling the Full Value of Connectivity - Ericsson"

The Ericsson website highlights the company's commitment to enhancing global connectivity through advanced technology solutions, particularly focusing on 5G and its applications across various sectors. Key offerings include:

- **5G Network Solutions**: Ericsson provides comprehensive services for service providers, including 5G RAN, Core, and Transport, as well as OSS/BSS automation and managed services.
- **Enterprise Solutions**: The company aims to support businesses in leveraging 5G, edge computing, and cloud technologies to improve their performance and sustainability.
- **Network Services**: Ericsson offers various network deployment and management services, emphasizing automation and AI for enhanced efficiency.

## Recent Highlights
- **5G in Healthcare**: Recent sessions illustrate how 5G can transform surgical planning and enhance cloud robotics in hospitals.
- **Investment in Japan**: Ericsson has announced the creation of up to 300 R&D jobs in Japan, showcasing the region's importance to its strategic operations.
- **Sustainability Commitment**: The company emphasizes the need for reduced energy consumption and carbon emissions in mobile networks as part of their sustainability goals.

The site also contains insights into next-generation technologies and reports that inform industry trends, such as the **Ericsson Mobility Report** and various white papers on advanced connectivity topics. 

Overall, the website positions Ericsson as a leader in connectivity innovation, emphasizing their role in enabling a sustainable and efficient digital future.